# CS5339 Project

# Data Cleaning

In [10]:
import pandas as pd


from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove
import re

def replace(file_path):
    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(re.sub("  " , " ", line))

    #Copy the file permissions from the old file to the new file
    copymode(file_path, abs_path)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

replace('aclImdb/test-neg.txt')
replace('aclImdb/train-pos.txt')
replace('aclImdb/test-pos.txt')
replace('aclImdb/test-neg.txt')
replace('aclImdb/train-unsup.txt')

train_neg = pd.read_csv('aclImdb/train-neg.txt', header = None, delimiter = "\n")
train_pos = pd.read_csv('aclImdb/train-pos.txt', header = None, delimiter = "\n")
train_unsup = pd.read_csv('aclImdb/train-unsup.txt', header = None, delimiter = "\n")
test_pos = pd.read_csv('aclImdb/test-pos.txt', header = None, delimiter = "\n")
test_neg = pd.read_csv('aclImdb/test-neg.txt', header = None, delimiter = "\n")


all_train = pd.concat([train_neg, train_pos, train_unsup], ignore_index=True)

all_train.head(10)

,0
0,story of a man who has unnatural feelings for ...
1,airport '77 starts as a brand new luxury 747 p...
2,this film lacked something i couldn't put my f...
3,"sorry everyone , , , i know this is suppose..."
4,when i was little my parents took me along to ...
5,""" it appears that many critics find the idea ..."
6,the second attempt by a new york intellectual ...
7,"i don't know who to blame , the timid writers..."
8,this film is mediocre at best . angie harmon ...
9,the film is bad . there is no other way to sa...


## Load dataset
https://ai.stanford.edu/~amaas/data/sentiment/

In [1]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order
with open('aclImdb/alldata-id.txt') as alldata:
    for line_no, line in enumerate(alldata):
        tokens = gensim.utils.to_unicode(line).split()
        words = tokens[1:]
        tags = [line_no] # `tags = [tokens[0]]` would also work at extra memory cost
        split = ['train','test','extra','extra'][line_no//25000]  # 25k train, 25k test, 50k unlabled data
        sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//12500] # [12.5K pos, 12.5K neg]*2 then unknown
        alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

unable to import 'smart_open.gcs', disabling that module


100000 docs: 25000 train-sentiment, 25000 test-sentiment


In [5]:
# Check first document 
doc_list[0]

SentimentDocument(words=['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', '.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'teachers', '"', '.', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'teachers', '"', '.', 'the', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', '.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'i', 'immediately', 'recalled', '.', '.', '.', '.', '.', '.', '.', '.', '.', 'at', '.', '

## Model

In [6]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
print(cores)
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),
]

# speed setup by sharing results of 1st model's vocabulary scan
simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template
print(simple_models[0])
for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

12


/usr/local/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12)
Doc2Vec(dbow,d100,n5,mc2,s0.001,t12)
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12)


## Prediction

In [9]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved

import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

## Training word vectors and doc vectors

In [ ]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list,total_examples=100000, epochs=epoch)
            duration = '%.1f' % elapsed()
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

        if ((epoch + 1) % 5) == 0 or epoch == 0:
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if infer_err < best_error[name + '_inferred']:
                best_error[name + '_inferred'] = infer_err
                best_indicator = '*'
            print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2020-04-17 17:41:25.914880
*0.497320 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12) 0.0s 0.4s
*0.502800 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12)_inferred 0.0s 5.5s
*0.497320 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t12) 0.0s 0.4s
*0.489200 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t12)_inferred 0.0s 1.9s
*0.497320 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12) 0.0s 0.7s
*0.499200 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12)_inferred 0.0s 2.9s
completed pass 1 at alpha 0.025000
*0.401880 : 2 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12) 27.2s 0.4s
*0.253160 : 2 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t12) 11.7s 0.4s
*0.262320 : 2 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12) 18.0s 0.4s
completed pass 2 at alpha 0.023800
*0.314000 : 3 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12) 51.3s 0.4s
